# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
!pip install --upgrade azureml-sdk
!pip install --upgrade azureml-core

     |████████████████████████████████| 136 kB 5.1 MB/s eta 0:00:01
     |████████████████████████████████| 8.6 MB 10.3 MB/s eta 0:00:01
     |████████████████████████████████| 3.3 MB 81.8 MB/s eta 0:00:01
     |████████████████████████████████| 38.2 MB 87.7 MB/s eta 0:00:01
     |████████████████████████████████| 247 kB 85.1 MB/s eta 0:00:01
     |████████████████████████████████| 69 kB 8.3 MB/s  eta 0:00:01
     |████████████████████████████████| 313 kB 86.4 MB/s eta 0:00:01
     |████████████████████████████████| 20.2 MB 82.2 MB/s eta 0:00:01
ERROR: azureml-widgets 1.49.0 has requirement azureml-core~=1.49.0, but you'll have azureml-core 1.51.0 which is incompatible.
ERROR: azureml-widgets 1.49.0 has requirement azureml-telemetry~=1.49.0, but you'll have azureml-telemetry 1.51.0 which is incompatible.
ERROR: azureml-training-tabular 1.49.0 has requirement azureml-automl-core~=1.49.0, but you'll have azureml-automl-core 1.51.0 which is incompatible.
ERROR: azureml-train 1.49.0 has re

In [2]:
import logging
import os
import csv

#from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
#from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

Title: Car Evaluation Database

Task: To create the hierarchical decision model

This model evaluates cars according to the following concept structure:

 CAR:car acceptability;

 buying:buying price;v-high, high, med, low
 
 maint:price of the maintenance;v-high, high, med, low
 
 doors:number of doors;2, 3, 4, 5-more
 
 persons:capacity in terms of persons to carry;2, 4, more
 
 lug_boot:the size of luggage boot;small, med, big
 
 safety:estimated safety of the car;low, med, high
  
Class Distribution;unacc,acc,good,v-good

In [3]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'auto-ml'
project_folder = './project'

experiment=Experiment(ws, experiment_name)

In [4]:
print(ws)

Workspace.create(name='quick-starts-ws-234786', subscription_id='610d6e37-4747-4a20-80eb-3aad70a55f43', resource_group='aml-quickstarts-234786')


In [5]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# Create the cluster
amlcompute_cluster_name = "auto-ml"

compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
    # for GPU, use "STANDARD_NC6"
    #vm_priority = 'lowpriority', # optional
    max_nodes=4)
compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded..........................................................................................................................................................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


In [6]:
# Create AML Dataset and register it into Workspace
key='car evaluation data set'
data = 'https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data'

df = pd.read_csv(data)
columns = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'class']
df.columns = columns

# Convert the DataFrame to a TabularDataset
dataset = Dataset.Tabular.register_pandas_dataframe(
    dataframe=df, 
    target=(ws.get_default_datastore(), key), 
    name=key, 
    description='Dataset description')

df.describe()


Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to car evaluation data set/1af3461c-63bc-4e8e-a517-d9b5ea913605/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


,buying,maint,doors,persons,lug_boot,safety,class
count,1727,1727,1727,1727,1727,1727,1727
unique,4,4,4,3,3,3,4
top,high,high,5more,more,big,high,unacc
freq,432,432,432,576,576,576,1209


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

Automl setting

 - experiment time out minutes; to avoid endless working

 - max concurrent iterations; between cost and performance

 - primary metric; AUC is a widely used metric to evaluate the performance of binary classifiers. The weighted version extends it to multi-class classification by considering the class imbalance when computing the average AUC across classes. 

Automl configuration

 - task = "classification"; To make classification model

 - label_column_name="class"; This column gives evalation result

 - enable_early_stopping= True; To save cost at score improvement small

 - featurization= 'auto'; Automated separation for training data and test data


In [7]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'
}
# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="class",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [8]:
# TODO: Submit your experiment

remote_run = experiment.submit(automl_config)


Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
auto-ml,AutoML_afa079e7-2e68-4d81-b05e-95a7c00c17db,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [9]:
from azureml.widgets import RunDetails

RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [10]:
#Wait for the AutoML run to complete
remote_run.wait_for_completion(show_output=True)

#"best_run" contains the model, metrics, and run ID
#"fitted_model" contains the actual trained model object
best_run, fitted_model = remote_run.get_output()

#"best_run_metrics" contains a dictionary of the metrics associated with the best run.
best_run_metrics = best_run.get_metrics()

# Display all the properties of the best model
print(best_run.get_properties())


Experiment,Id,Type,Status,Details Page,Docs Page
auto-ml,AutoML_afa079e7-2e68-4d81-b05e-95a7c00c17db,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  In order to accurately evaluate the model(s) trained by AutoML, we leverage a dataset that the model is not trained on. Hence, if the user doesn't provide an explicit validation dataset, a part of the training dataset is used to achieve this. For smaller datasets (fewer than 20,000 samples), cross-validation is leveraged, else a single hold-out set is split from the training data to serve as the validation dataset. Hence, for your input data we leverage cross-validation with 10 folds, if the number of training samples are fewer than 1000, and 3 folds in all other cases.
              Learn more about cross validation: https://aka.ms/AutomatedMLCrossValidation
DETAILS:      
+------------------------------+
|Number of folds               |
+==============================+
|3                             |
+------

Package:azureml-automl-runtime, training version:1.50.0, current version:1.49.0
Package:azureml-core, training version:1.50.0, current version:1.49.0
Package:azureml-dataprep, training version:4.10.6, current version:4.9.1
Package:azureml-dataprep-rslex, training version:2.17.5, current version:2.16.1
Package:azureml-dataset-runtime, training version:1.50.0, current version:1.49.0
Package:azureml-defaults, training version:1.50.0, current version:1.49.0
Package:azureml-interpret, training version:1.50.0, current version:1.49.0
Package:azureml-mlflow, training version:1.50.0, current version:1.49.0
Package:azureml-pipeline-core, training version:1.50.0.post1, current version:1.49.0
Package:azureml-responsibleai, training version:1.50.0, current version:1.49.0
Package:azureml-telemetry, training version:1.50.0, current version:1.49.0
Package:azureml-train-automl-client, training version:1.50.0, current version:1.49.0
Package:azureml-train-automl-runtime, training version:1.50.0, current 

In [11]:
#TODO: Save the best model
# Save the fitted model as a pickle file
import pickle
filename = 'best_model.pkl'
with open(filename, 'wb') as file:
    pickle.dump(fitted_model, file)


In [12]:
print(filename)

best_model.pkl


In [13]:
import pickle
with open(filename, 'rb') as file:
    loaded_model=pickle.load(file)
print(loaded_model)

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mounts/clusters/notebook234786/code/Users/odl_user_234786')),
                ('MaxAbsScaler', MaxAbsScaler(copy=True)),
                ('LightGBMClassifier',
                 LightGBMClassifier(min_data_in_leaf=20, n_jobs=1, problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=None))],
         verbose=False)
Y_transformer(['LabelEncoder', LabelEncoder()])


## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [15]:
best_run.register_model(model_name='best_aml_model', model_path='./outputs/')

Model(workspace=Workspace.create(name='quick-starts-ws-234786', subscription_id='610d6e37-4747-4a20-80eb-3aad70a55f43', resource_group='aml-quickstarts-234786'), name=best_aml_model, id=best_aml_model:1, version=1, tags={}, properties={})

In [16]:
from azureml.core import Model

model = Model.register(workspace=ws, 
    model_path='best_model.pkl', 
    model_name='best_aml_model')


Registering model best_aml_model


In [17]:
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment

In [21]:
! pip list

Package                                 Version
--------------------------------------- -------------------
absl-py                                 0.15.0
adal                                    1.2.7
adlfs                                   2023.1.0
aiohttp                                 3.8.4
aiohttp-cors                            0.7.0
aiosignal                               1.3.1
ansiwrap                                0.8.4
antlr4-python3-runtime                  4.9.3
anyio                                   3.6.2
applicationinsights                     0.11.10
arch                                    4.14
argcomplete                             2.0.0
argon2-cffi                             21.3.0
argon2-cffi-bindings                    21.2.0
arrow                                   1.2.3
arviz                                   0.11.2
astroid                                 2.15.2
asttokens                               2.2.1
astunparse                              1.6.3
async-tim

In [18]:
#from azureml.core import InferenceConfig, Environment

env = Environment.get(workspace=ws, name='quick-starts-ws-234786')
inference_config = InferenceConfig(entry_script='score.py', environment=env) #####

Exception: Error retrieving the environment definition. Code: 404
: {
  "error": {
    "code": "UserError",
    "severity": null,
    "message": "No environment exists for name: quick-starts-ws-234786, version: , label: ",
    "messageFormat": null,
    "messageParameters": null,
    "referenceCode": null,
    "detailsUri": null,
    "target": null,
    "details": [],
    "innerError": {
      "code": "NotFoundError",
      "innerError": null
    },
    "debugInfo": null,
    "additionalInfo": null
  },
  "correlation": {
    "operation": "87af802c69b8b7c135ec279c0cdacc22",
    "request": "3c65792cbc9a42cc"
  },
  "environment": "southcentralus",
  "location": "southcentralus",
  "time": "2023-06-01T05:20:05.5394671+00:00",
  "componentName": "environment-management"
}

In [ ]:
from azureml.core.webservice import AciWebservice

deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)
service = Model.deploy(workspace=ws,
                       name='automl-deploy',
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=deployment_config,
                       overwrite=True)
                       
service.wait_for_deployment(show_output=True)


TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
